In [1]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-4-25 torch 1.11.0+cu113 CUDA:0 (Tesla K80, 11441MiB)



  0%|          | 0.00/40.8M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Extracting frames from the video

import cv2

frameWidth = 480
frameHeight = 320

cap = cv2.VideoCapture("/content/video/train030.mp4")

i = 0
success, img1 = cap.read()

while success:
    img = cv2.resize(img1, (frameWidth, frameHeight))
    # cv2.imshow("Result", img)
    cv2.imwrite(f'/content/frames/{i}_frame.jpg', img)
    if cv2.waitKey(1) and 0xFF == ord('q'):
         break
    i += 1
    success, img1 = cap.read()

cap.release()

In [6]:
# Testing model on one image

img_path = '/content/frames/101_frame.jpg'
img = cv2.imread(img_path)
results = model(img) # giving input image to the model
print(results.__dict__.keys())

dict_keys(['imgs', 'pred', 'names', 'files', 'times', 'xyxy', 'xywh', 'xyxyn', 'xywhn', 'n', 't', 's'])


In [7]:
# Coordinated for bounding box (top left, bottom right)
# results.names is the label map
results.xyxy

#         x1           y1           x2           y2         confidence_score  label

[tensor([[2.74469e+02, 1.42537e+02, 2.89463e+02, 1.86505e+02, 8.67550e-01, 0.00000e+00],
         [2.07590e+02, 1.82581e+02, 2.29196e+02, 2.35566e+02, 8.51232e-01, 0.00000e+00],
         [2.97266e+02, 8.88963e+01, 3.13737e+02, 1.32790e+02, 8.34315e-01, 0.00000e+00],
         [3.50582e+02, 6.44135e+01, 3.68792e+02, 1.05645e+02, 8.19764e-01, 0.00000e+00],
         [1.54924e+02, 2.29925e+02, 1.76646e+02, 2.87341e+02, 8.18921e-01, 0.00000e+00],
         [3.06487e+00, 2.47888e+02, 2.38009e+01, 3.05461e+02, 8.03109e-01, 0.00000e+00],
         [3.36095e+02, 6.69925e+01, 3.51076e+02, 1.05270e+02, 7.60462e-01, 0.00000e+00],
         [7.73887e+01, 2.60360e+02, 9.93548e+01, 3.19657e+02, 7.48011e-01, 0.00000e+00],
         [2.42373e+01, 2.97008e+02, 5.29570e+01, 3.19639e+02, 7.02741e-01, 0.00000e+00],
         [1.01847e+02, 2.78487e+02, 1.27749e+02, 3.19738e+02, 6.98597e-01, 0.00000e+00],
         [2.33686e+02, 1.12361e+02, 2.49951e+02, 1.55118e+02, 6.24973e-01, 0.00000e+00],
         [1.39692e+02

In [8]:
# For help

# help(results)

In [9]:
bboxes = results.xyxy[0].detach().cpu().numpy()

# Detach tensor from cuda and put those tensor on CPU
# Convert those tensor values to numpy

In [10]:
bboxes

array([[     274.47,      142.54,      289.46,      186.51,     0.86755,           0],
       [     207.59,      182.58,       229.2,      235.57,     0.85123,           0],
       [     297.27,      88.896,      313.74,      132.79,     0.83431,           0],
       [     350.58,      64.413,      368.79,      105.65,     0.81976,           0],
       [     154.92,      229.92,      176.65,      287.34,     0.81892,           0],
       [     3.0649,      247.89,      23.801,      305.46,     0.80311,           0],
       [      336.1,      66.992,      351.08,      105.27,     0.76046,           0],
       [     77.389,      260.36,      99.355,      319.66,     0.74801,           0],
       [     24.237,      297.01,      52.957,      319.64,     0.70274,           0],
       [     101.85,      278.49,      127.75,      319.74,      0.6986,           0],
       [     233.69,      112.36,      249.95,      155.12,     0.62497,           0],
       [     139.69,      149.62,      156.

In [12]:
# Plotting bounding box on that image
# Plotting the confidence score 
import os

canvas = img.copy()

for bbox in bboxes:
    x1, y1, x2, y2, score, label_id = bbox
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # converting coordinated to integer
    label = results.names[int(label_id)]

    cv2.rectangle(canvas, (x1, y1), (x2, y2), (0, 255, 0), 1)
    cv2.putText(canvas, 
                f'{label}-{score:.2f}', 
                (x1, y1), 
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255,255,255),
                1)
    
cv2.imwrite(os.path.join('/content/output', 'Output_' + os.path.basename(img_path)), canvas)

True

In [21]:
#### For Video

In [13]:
import cv2
import numpy as np

def get_predictions(img):
    results = model(img)
    bboxes = results.xyxy[0].detach().cpu().numpy()
    return bboxes, results.names

def draw_bboxes(img, bboxes, label_map):
    canvas = img.copy()
    for bbox in bboxes:
        x1, y1, x2, y2, score, label_id = bbox
        if(label_id == 0):
          x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
          label = label_map[int(label_id)]


          cv2.rectangle(canvas, (x1, y1), (x2, y2), (0, 255, 0), 1)   # Create bounding box
          cv2.putText(canvas,     # Write name of detected object (label) and confidence score
                    f'{label}-{score:.2f}', 
                    (x1, y1), # Change y1 to change the display of label abpve bpunding box
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.1,
                    (255,255,255),
                    1)
    return canvas



def export_video(video_path, output_path):

    # Create a VideoCapture object
    cap = cv2.VideoCapture(video_path)    # Reading a video for that video path

    # Check if camera opened successfully
    if (cap.isOpened() == False): 
        print("Unable to read camera feed")
    # Default resolutions of the frame are obtained.The default resolutions are system dependent.
    # We convert the resolutions from float to integer.
    # Taking the same width and height of the original video to keep consistent results
    frame_width = int(cap.get(3))   # 3 - Width
    frame_height = int(cap.get(4))  # 4 - height  5 - fps
    fps = 15  # Having same FPS is important to keep consistent frames

    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec
    # fourcc has values configurations for that codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    # out is keeping track of all the frames and storing video

    while(True):
        ret, frame = cap.read()
        if ret == True:    # to check is frame is valid

            bboxes, label_map = get_predictions(frame)
            plotted_bboxes_img = draw_bboxes(frame, bboxes, label_map)
            out.write(plotted_bboxes_img) # Adding frame one by one

        # Break the loop
        else:
            break  

    # When everything done, release the video capture and video write objects
    cap.release()
    out.release()

    # Closes all the frames
    # cv2.destroyAllWindows()

In [14]:
video_path = '/content/video/train030.mp4'
output_path = '/content/output/train030_output.mp4'

export_video(video_path, output_path)